In [869]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt

In [871]:
# Load xor toy dataset
with open('/Users/aton/Machine Learning/5.2 Data/xordata.pkl', 'rb') as f:
    data = pickle.load(f)

X_train = data['X_train'] # 800 training data points with 2 features
y_train = data['y_train'] # training binary labels {0,1}

X_test = data['X_test']
y_test = data['y_test']

In [873]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(800, 2) (800,)
(200, 2) (200,)


In [875]:
#plt.scatter(X_train[:,0], X_train[:,1], s=40, c=y_train, cmap=plt.cm.Spectral)

In [877]:
# Normalize data (maybe make a method instead)
X = X_train
x1sum=0
x2sum=0
for i in range(X.shape[0]):
    xn = X[i]
    x1 = xn[0]
    x2 = xn[1]
    x1sum= x1sum + x1
    x2sum = x2sum + x2
x1avr = x1sum/X.shape[0]
x2avr = x2sum/X.shape[0]
x1var=0
x2var=0
for i in range(X.shape[0]):
    xn = X[i]
    x1 = xn[0]
    x2 = xn[1]
    x1var = x1var + (x1-x1avr)**2
    x2var = x2var + (x2-x2avr)**2

Xnorm = X
for i in range(X.shape[0]):
    Xnorm[i,0] = (X[i,0]-x1avr)/math.sqrt(x1var)
    Xnorm[i,1] = (X[i,1]-x2avr)/math.sqrt(x2var)


In [949]:
# Random initialization
upperlowerbound = 5
M=10
w0 = np.random.uniform(low=-upperlowerbound, high=upperlowerbound, size=(M, 2))
w1 = np.random.uniform(low=-upperlowerbound, high=upperlowerbound, size=(M+1, 1))
z0 = 1

In [881]:
# NN - first layer
def NN01(x,w0,z0,w1):  # x = 1x2, w0 = 4x2, z0 = c, w1 = 5x1
    a = w0 @ x.T
    a = a.flatten()
    z = np.array([ReLu(val) for val in a])       
    z = np.insert(z,0,z0)
    return NN02(z,w1,a)

In [883]:
## NN - second layer
def NN02(z,w1,a1):
    a = w1.T @ z
    y = sigmoid(a)
    if(y<1/2):
        c=0
    else:
        c=1
    return (y,a1,z)

In [885]:
# ReLu function
def ReLu(x):
    if(x>0):
        return x
    else:
        return 0

In [887]:
# derivative of ReLu
def dReLu(x):
    if(x>0):
        return 1
    else:
        return 0

In [889]:
# Sigmoid function
def sigmoid(x):
    return 1/(1+math.exp(-x))

In [923]:
# Forward Propagation
def FP(Vector):
    output = []
    for val in Vector:
        a,b,c = NN01(val,w0,z0,w1)
        #if(a<1/2):
        #    a=0.000001
        #else:
        #    a=0.999999
        output.append(a)
    output = np.array(output)
    return output

In [931]:
# Loss function
def CrossEntropyLoss(y, X):
    y = np.array(y)
    X = np.array(X)
    sum = 0
    for i in range(X.shape[0]):
        sum = sum + y[i]*math.log(X[i]+0.000001) + (1-y[i])*math.log(1-X[i]+0.000001)
    sum = (-1 * sum)/X.shape[0]
    return sum

In [895]:
# backprop

def backprop(x,y,X,a1,z1,w0,w1):
    deltak = 0
    #for i in range(X.shape[0]):
        #deltak = deltak + X[i] - y[i]
    deltak = deltak + X - y
    
    dEdw1 = deltak * z1
    
    delta1 = []
    for i in range(a1.shape[0]):
        delta1.append(dReLu(a1[i])*deltak*w1[i+1])
    
    delta1 = np.array(delta1)
    dEdw0 = delta1 * x

    return (dEdw1, dEdw0)

In [897]:
iterations = 7000
batch = 50
stepsize = 0.01

In [953]:
# SGD
for iteration in range(iterations):
    indices = np.random.permutation(Xnorm.shape[0])  # random seed
    Xnorm = Xnorm[indices]  # Shuffle rows according to seed
    y_train = y_train[indices]  # the same but for y

    for i in range(batch):
        xn = Xnorm[i]
        t = y_train[i]
        y,a1,z1 = NN01(xn,w0,z0,w1)

        dEdw1, dEdw0 = backprop(xn,t,y,a1,z1,w0,w1)

        for i in range(w0.shape[0]):
            w0[i,0] = w0[i,0] - stepsize*dEdw0[i,0]
            w0[i,1] = w0[i,1] - stepsize*dEdw0[i,1]

        for i in range(w1.shape[0]):
            w1[i,] = w1[i,] - stepsize*dEdw1[i,]



In [957]:
#test
xxx = FP(Xnorm)
print(CrossEntropyLoss(y_train,xxx))

0.09772099697493836
